In [101]:
from bs4 import BeautifulSoup,Comment
import urllib.request
import pandas as pd
import requests
import numpy as np

In [81]:
#to get soup for 2020 team offense using Comment, used below
url = "https://www.pro-football-reference.com/years/2020/"
soup = BeautifulSoup(requests.get(url).content, 'html.parser')
table = soup.select_one('#all_team_stats').find_next(text=lambda t: isinstance(t, Comment))
table = BeautifulSoup(table, 'html.parser')



In [82]:
#what the HTML looks like
table


<div class="table_container" id="div_team_stats">
<table class="per_match_toggle sortable stats_table" data-cols-to-freeze=",2" id="team_stats">
<caption>Team Offense Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr class="over_header">
<th aria-label="" class="over_header center" colspan="2" data-stat=""></th><th></th><th></th><th></th>
<th aria-label="" class="over_header center" colspan="3" data-stat="header_tot_off_tgl">Tot Yds &amp; TO</th><th></th><th></th>
<th aria-label="" class="over_header center" colspan="7" data-stat="header_pass">Passing</th>
<th aria-label="" class="over_header center" colspan="5" data-stat="header_rush">Rushing</th>
<th aria-label="" class="over_header center" colspan="3" data-stat="header_penalties">Penalties</th>
<th aria-label="" class="over_header center" colspan="2" data-stat=""></t

In [124]:
#function to rename some columns and columns that are missing titles from HTML table
def rename_cols(df):
    dict = {'Unnamed: 0_level_0':'TotOff',
            'Unnamed: 1_level_0':'TotOff',
            'Unnamed: 2_level_0':'TotOff',
            'Unnamed: 3_level_0':'TotOff',
            'Unnamed: 4_level_0':'TotOff',
            'Tot Yds & TO':'TotOff',
            'Tot Yds & TO':'TotOff',
            'Tot Yds & TO':'TotOff',
            'Unnamed: 8_level_0':'TotOff',
            'Unnamed: 9_level_0':'TotOff',
            'Unnamed: 25_level_0':'Score%',
            'Unnamed: 26_level_0':'TurnOver%',
            'Unnamed: 27_level_0':'EXP',}
        
    # call rename() method
    df.rename(columns=dict,inplace=True)
    return(df)


In [111]:
#for loop to get data for X years

year = 2020
url = "https://www.pro-football-reference.com/years/"
urls = []
OFFdfs = []
for index in np.arange(2):
    #print(str(year - index))
    OFFdfs.append(year - index)
    urls.append(url + str(year - index) +'/')
    
#print(urls)
#OFFdfs = []
count=0
for index in urls:
    soup = BeautifulSoup(requests.get(index).content, 'html.parser')
    table = soup.select_one('#all_team_stats').find_next(text=lambda t: isinstance(t, Comment))
    table = BeautifulSoup(table, 'html.parser')
    df = pd.read_html(str(table))[0]
    OFFdfs[count] = df
    count+=1



In [123]:
#for 2 years, 2020 and 2019, data frames for Team Offense
OFF_2020 = pd.DataFrame(OFFdfs[0])
rename_cols(OFF_2020)
OFF_2019 = pd.DataFrame(OFFdfs[1])
rename_cols(OFF_2019)



TotOff                                                                      \
       Rk                    Tm     G       PF       Yds      Ply  Y/P     TO   
0     1.0      Baltimore Ravens  16.0    531.0    6521.0   1064.0  6.1   15.0   
1     2.0   San Francisco 49ers  16.0    479.0    6097.0   1012.0  6.0   23.0   
2     3.0  Tampa Bay Buccaneers  16.0    458.0    6366.0   1086.0  5.9   41.0   
3     4.0    New Orleans Saints  16.0    458.0    5982.0   1011.0  5.9    8.0   
4     5.0    Kansas City Chiefs  16.0    451.0    6067.0    976.0  6.2   15.0   
5     6.0        Dallas Cowboys  16.0    434.0    6904.0   1069.0  6.5   18.0   
6     7.0  New England Patriots  16.0    420.0    5664.0   1095.0  5.2   15.0   
7     8.0     Minnesota Vikings  16.0    407.0    5656.0    970.0  5.8   20.0   
8     9.0      Seattle Seahawks  16.0    405.0    5991.0   1046.0  5.7   20.0   
9    10.0      Tennessee Titans  16.0    402.0    5805.0    949.0  6.1   17.0   
10   11.0      Los Angeles Rams  16.0    394.0    5998.0   1055.0  5.7   24.0   
11   12.0   Philadelphia Eagles  16.0    385.0    5772.0   1104.0  5.2   23.0   
12   13.0       Atlanta Falcons  16.0    381.0    6075.0   1096.0  5.5   25.0   
13   14.0        Houston Texans  16.0    378.0    5792.0   1017.0  5.7   22.0   
14   15.0     Green Bay Packers  16.0    376.0    5528.0   1020.0  5.4   13.0   
15   16.0     Arizona Cardinals  16.0    361.0    5467.0   1000.0  5.5   18.0   
16   17.0    Indianapolis Colts  16.0    361.0    5238.0   1016.0  5.2   21.0   
17   18.0         Detroit Lions  16.0    341.0    5549.0   1021.0  5.4   23.0   
18   19.0       New York Giants  16.0    341.0    5416.0   1012.0  5.4   33.0   
19   20.0     Carolina Panthers  16.0    340.0    5469.0   1077.0  5.1   35.0   
20   21.0  Los Angeles Chargers  16.0    337.0    5879.0    997.0  5.9   31.0   
21   22.0      Cleveland Browns  16.0    335.0    5455.0    973.0  5.6   28.0   
22   23.0         Buffalo Bills  16.0    314.0    5283.0   1018.0  5.2   19.0   
23   24.0       Oakland Raiders  16.0    313.0    5819.0    989.0  5.9   17.0   
24   25.0        Miami Dolphins  16.0    306.0    4960.0   1022.0  4.9   26.0   
25   26.0  Jacksonville Jaguars  16.0    300.0    5468.0   1020.0  5.4   20.0   
26   27.0   Pittsburgh Steelers  16.0    289.0    4428.0    937.0  4.7   30.0   
27   28.0        Denver Broncos  16.0    282.0    4777.0    954.0  5.0   16.0   
28   29.0         Chicago Bears  16.0    280.0    4749.0   1020.0  4.7   19.0   
29   30.0    Cincinnati Bengals  16.0    279.0    5169.0   1049.0  4.9   30.0   
30   31.0         New York Jets  16.0    276.0    4368.0    956.0  4.6   25.0   
31   32.0   Washington Redskins  16.0    266.0    4395.0    885.0  5.0   21.0   
32    NaN              Avg Team   NaN    365.0    5565.8   1016.1  5.5   22.2   
33    NaN          League Total   NaN  11680.0  178107.0  32516.0  5.5  711.0   
34    NaN              Avg Tm/G   NaN     22.8     347.9     63.5  5.5    1.4   

                    ...  Rushing                     Penalties           \
       FL     1stD  ...      Yds     TD  Y/A    1stD       Pen      Yds   
0     7.0    386.0  ...   3296.0   21.0  5.5   188.0     109.0    867.0   
1    10.0    336.0  ...   2305.0   23.0  4.6   110.0     105.0    939.0   
2    11.0    353.0  ...   1521.0   15.0  3.7    81.0     133.0   1111.0   
3     2.0    347.0  ...   1738.0   12.0  4.3    97.0     120.0   1036.0   
4    10.0    350.0  ...   1569.0   16.0  4.2    93.0     107.0   1029.0   
5     7.0    379.0  ...   2153.0   18.0  4.8   120.0     109.0   1008.0   
6     6.0    338.0  ...   1703.0   17.0  3.8   110.0      94.0    828.0   
7    12.0    314.0  ...   2133.0   19.0  4.5   106.0      96.0    895.0   
8    14.0    341.0  ...   2200.0   15.0  4.6   121.0     109.0    882.0   
9     9.0    317.0  ...   2223.0   21.0  5.0   104.0      99.0    932.0   
10    7.0    342.0  ...   1499.0   20.0  3.7    92.0     118.0    899.0   
11   15.0    354.0  ...   1939